In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

2024-11-18 20:30:10.063183: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-18 20:30:10.316538: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-18 20:30:10.385862: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-18 20:30:10.563372: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-18 20:30:10.600687: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-18 20:30:10.800481: I tensorflow/core/platform/cpu_feature_gu

In [2]:
file_path = ['/home/genty/Hiwi-TimeSeries/Hiwi-TimeSeries/dataset/DatAmount_dataset_new/1_CMX/CMX1_AL_CP1.csv',
             '/home/genty/Hiwi-TimeSeries/Hiwi-TimeSeries/dataset/DatAmount_dataset_new/1_CMX/CMX1_AL_CP2.csv',
             '/home/genty/Hiwi-TimeSeries/Hiwi-TimeSeries/dataset/DatAmount_dataset_new/1_CMX/CMX1_S_CP1.csv',
             '/home/genty/Hiwi-TimeSeries/Hiwi-TimeSeries/dataset/DatAmount_dataset_new/1_CMX/CMX1_S_CP2.csv',
             '/home/genty/Hiwi-TimeSeries/Hiwi-TimeSeries/dataset/DatAmount_dataset_new/2_DMC/DMC2_AL_CP1.csv',
             '/home/genty/Hiwi-TimeSeries/Hiwi-TimeSeries/dataset/DatAmount_dataset_new/2_DMC/DMC2_AL_CP2.csv',
             '/home/genty/Hiwi-TimeSeries/Hiwi-TimeSeries/dataset/DatAmount_dataset_new/2_DMC/DMC2_S_CP1.csv',
             '/home/genty/Hiwi-TimeSeries/Hiwi-TimeSeries/dataset/DatAmount_dataset_new/2_DMC/DMC2_S_CP2.csv']

In [15]:
features =  ['LOAD|1', 'LOAD|2', 'LOAD|3', 'LOAD|6',
       'ENC_POS|1', 'ENC_POS|2', 'ENC_POS|3','ENC_POS|6',
       'CTRL_DIFF2|1', 'CTRL_DIFF2|2', 'CTRL_DIFF2|3', 'CTRL_DIFF2|6',
       'TORQUE|1', 'TORQUE|2', 'TORQUE|3', 'TORQUE|6',
       'DES_POS|1', 'DES_POS|2', 'DES_POS|3', 'DES_POS|6',
       'CTRL_DIFF|1', 'CTRL_DIFF|2', 'CTRL_DIFF|3' ,'CTRL_DIFF|6',
       'CTRL_POS|1', 'CTRL_POS|2', 'CTRL_POS|3', 'CTRL_POS|6',
       'VEL_FFW|1', 'VEL_FFW|2','VEL_FFW|3', 'VEL_FFW|6',
       'CONT_DEV|1','CONT_DEV|2', 'CONT_DEV|3', 'CONT_DEV|6',
       'CMD_SPEED|1', 'CMD_SPEED|2', 'CMD_SPEED|3', 'CMD_SPEED|6',
       'TORQUE_FFW|1', 'TORQUE_FFW|2', 'TORQUE_FFW|3', 'TORQUE_FFW|6',
       'ENC1_POS|1', 'ENC1_POS|2', 'ENC1_POS|3','ENC1_POS|6',
       'ENC2_POS|1', 'ENC2_POS|2', 'ENC2_POS|3', 'ENC2_POS|6']

feature_subset = ['TORQUE|6', 'LOAD|6', 'CMD_SPEED|6', 'VEL_FFW|6',  'VEL_FFW|1', 'VEL_FFW|2']

target = 'CURRENT|6'

In [4]:
output_folder = '/home/genty/Hiwi-TimeSeries/Hiwi-TimeSeries/output/8dataset/'
#result_df = pd.DataFrame(columns=['Features', 'Test_R2', 'Test_RMSE'])
checkpoint = pd.DataFrame(columns=['Features', 'Current_Best', 'Test_RMSE', 'Test_R2'])
file = file_path[3]
filenamefinal = os.path.join(output_folder, f'FS_{os.path.basename(file)}.csv')
result_df = pd.read_csv(filenamefinal)
checkpoint_filename = os.path.join(output_folder, f'checkpoint.csv')
#checkpoint = pd.read_csv(checkpoint_filename)

In [5]:
def normalize(X, y):
    scaler_X = MinMaxScaler(feature_range=(0, 1))
    scaler_y = MinMaxScaler(feature_range=(0, 1))
    scaled_X = scaler_X.fit_transform(X)
    scaled_y = scaler_y.fit_transform(y)

    return scaler_X, scaler_y, scaled_X, scaled_y

In [6]:
def denormalize(train_predict, y_train, test_predict, y_test, scaler_y):
    train_predict_inv = scaler_y.inverse_transform(train_predict)
    y_train_inv = scaler_y.inverse_transform(y_train)
    test_predict_inv = scaler_y.inverse_transform(test_predict)
    y_test_inv = scaler_y.inverse_transform(y_test)
    
    return train_predict_inv, y_train_inv, test_predict_inv, y_test_inv

In [7]:
def create_dataset(X, y, time_step=60):
    Xs, ys = [], []
    for i in range(len(X) - time_step):
        if isinstance(X, np.ndarray):
            Xs.append(X[i:(i + time_step)])
            ys.append(y[i + time_step])
        else:
            Xs.append(X.iloc[i:(i + time_step)].values)
            ys.append(y.iloc[i + time_step].values)
    return np.array(Xs), np.array(ys)

In [8]:
def build_model(input_shape):
    model = Sequential()
    model.add(LSTM(64, return_sequences=True, input_shape=input_shape))
    model.add(LSTM(32, return_sequences=False))
    model.add(Dense(25))
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    return model

In [9]:
def run_lstm(X, y):
    scaler_X, scaler_y, scaled_X, scaled_y = normalize(X, y)
    X = scaled_X
    y = scaled_y
    train_size = int(len(X) * 0.7)
    test_size = len(X) - train_size
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    time_step = 60
    X_train, y_train = create_dataset(X_train, y_train, time_step)
    X_test, y_test = create_dataset(X_test, y_test, time_step)

    input_shape = (X_train.shape[1], X_train.shape[2])
    model = build_model(input_shape)
    history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_test, y_test), verbose=0)
    train_predict = model.predict(X_train)
    test_predict = model.predict(X_test)

    train_predict, y_train, test_predict, y_test = denormalize(train_predict, y_train, test_predict, y_test, scaler_y)
    return train_predict, y_train, test_predict, y_test

In [16]:
selected_features = ['TORQUE|6', 'LOAD|6', 'TORQUE_FFW|6']
best_performance = float('inf')


In [20]:
df = pd.read_csv(file)
y = df[[target]]
current_features = set(selected_features + feature_subset + ['ENC2_POS|1']) 
        
print(current_features)
X_subset = df[current_features]
train_predict, y_train, test_predict, y_test = run_lstm(X_subset, y)

train_r2 = r2_score(y_train, train_predict)
test_r2 = r2_score(y_test, test_predict)
train_rmse = np.sqrt(mean_squared_error(y_train, train_predict))
test_rmse = np.sqrt(mean_squared_error(y_test, test_predict))
print(test_rmse)

result_df = result_df.append({
            'Features': current_features,
            'Test_R2': test_r2,
            'Test_RMSE': test_rmse
            }, ignore_index=True)

result_df.to_csv(filenamefinal, index=False)

{'TORQUE_FFW|6', 'TORQUE|6', 'VEL_FFW|1', 'CMD_SPEED|6', 'VEL_FFW|2', 'ENC2_POS|1', 'LOAD|6', 'VEL_FFW|6'}


/home/genty/.local/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


993/993 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step
425/425 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step
0.10944114978948537


In [11]:
df = pd.read_csv(file)
y = df[[target]]

for i in range(4,len(features)):
        print("Outer loop:", i+1)
        current_best_performance = float('inf')
        best_feature = None
        for f in features[39:]:
                print(f)
                if f not in selected_features:
                        current_features = selected_features + [f]
                        X_subset = df[current_features]
                        train_predict, y_train, test_predict, y_test = run_lstm(X_subset, y)

                        train_r2 = r2_score(y_train, train_predict)
                        test_r2 = r2_score(y_test, test_predict)
                        train_rmse = np.sqrt(mean_squared_error(y_train, train_predict))
                        test_rmse = np.sqrt(mean_squared_error(y_test, test_predict))
                        print(test_rmse)

                        if test_rmse < current_best_performance:
                                current_best_performance = test_rmse
                                best_feature = f
                                checkpoint = checkpoint.append({
                                        'Features': current_features,
                                        'Current_Best': best_feature,
                                        'Test_RMSE': test_rmse,
                                        'Test_R2': test_r2
                                }, ignore_index=True)
                                checkpoint.to_csv(checkpoint_filename, index=False)
                                
        checkpoint = pd.read_csv(checkpoint_filename)
        #checkpoint = pd.read_csv(checkpoint_filename)
        min_rmse_row = checkpoint.loc[checkpoint['Test_RMSE'].idxmin()]
        best_feature = min_rmse_row['Current_Best']
        selected_features.append(best_feature)
        best_performance = min_rmse_row['Test_RMSE']
        result_df = result_df.append({
                        'Features': selected_features,
                        'Test_R2': min_rmse_row['Test_R2'],
                        'Test_RMSE': best_performance
                        }, ignore_index=True)

        result_df.to_csv(filenamefinal, index=False)

Outer loop: 5
CMD_SPEED|6


/home/genty/.local/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


993/993 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step
425/425 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step
0.109949515798555
TORQUE_FFW|1


/home/genty/.local/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


993/993 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step
425/425 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step
0.07913112792648323
TORQUE_FFW|2


...

993/993 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step
425/425 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step
0.08346056956508588
ENC2_POS|6


/home/genty/.local/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


993/993 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step
425/425 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step
0.10011721559545748
Outer loop: 6
CMD_SPEED|6


/home/genty/.local/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


KeyboardInterrupt: 